### Working in an investment company, I check at the end of every month if there are holdings in the portfolio that account for 2% or more of the total portfolio value, and the total portfolio size is greater than 300M ILS.

### There are two CSV files downloaded from the bank system, located in the same folder as this Python file.
### We will use Pandas to extract the Data Frame from the CSV files, clean the data, and perform some calculations to obtain the final result.

### Finally, a CSV file will be sent to an email address of our choice.
<hr>

In [1]:
import pandas as pd
import glob
import os
import smtplib
from email.message import EmailMessage
import datetime as dt

In [2]:
portfolio_value_file = glob.glob('BAMARIKBAKAR*.csv')
holdings_file = glob.glob('KRANOT_ITROT*.csv')

###  We want to screen out accounts that are smaller than 300M ILS and accounts with the name 'mehaka' included in their name. Additionally, we want to merge the two data frames into one.

In [3]:
portfolio_value = pd.read_csv(portfolio_value_file[0], encoding="cp1255",
                   usecols=['שם עברי קצר', 'מספר נייר', 'שווי נטו'])

In [4]:
portfolio_value.rename(columns = {'מספר נייר' : 'קרן'}, inplace=True)

In [5]:
portfolio_value = portfolio_value[(portfolio_value['שם עברי קצר']!="מחקה") & (portfolio_value["שווי נטו"] > 300000000)]

In [6]:
portfolio_holdings = pd.read_csv(holdings_file[0], encoding="cp1255", usecols=['קרן', 'שם קרן', 'מספר נייר',
                                                                                     'שם נייר/סוג חשבון', 'סה"כ ש"ח'])

In [7]:
data = portfolio_holdings.merge(right=portfolio_value.drop(columns=['שם עברי קצר']), how='left', on='קרן', sort = True)

### Add a column with a calculation of the value of a holding divided by the value of the portfolio. Additionally, we want to screen only holdings that are 2% or higher of the portfolio.

In [8]:
data['אחוז מהקרן'] = data['סה"כ ש"ח']/data['שווי נטו']*100

In [9]:
data_2 = data[(data["אחוז מהקרן"] > 2.00) & (data['מספר נייר']!=0)].copy()

In [10]:
data_2.head()

,קרן,שם קרן,מספר נייר,שם נייר/סוג חשבון,"סה""כ ש""ח",שווי נטו,אחוז מהקרן
12,1142556,חשבון פיקטיבי טכ עילית,69690,NVMI,11516785.73,540024469.0,2.132641
13,1142556,חשבון פיקטיבי טכ עילית,71670,CAMT,15755045.61,540024469.0,2.917469
17,1142556,חשבון פיקטיבי טכ עילית,445015,מטריקס,11881737.21,540024469.0,2.200222
19,1142556,חשבון פיקטיבי טכ עילית,720011,אנלייט אנרגיה,13305257.23,540024469.0,2.463825
25,1142556,חשבון פיקטיבי טכ עילית,1123355,אנרג'יקס,12313828.80,540024469.0,2.280235


### Save the file locally and send an email

In [11]:
data_2.to_csv("data_2.csv", encoding="cp1255")

In [17]:
os.environ['MY_MAIL'] = 'Your_email'
os.environ['MY_PASS'] = 'Your_password'

In [18]:
EMAIL_ADDRESS = os.environ.get("MY_MAIL")
EMAIL_PASSWORD = os.environ.get("MY_PASS")

In [13]:
to_email = input("Please insert an Email: ")

Please insert an Email: some_email@site.com


In [19]:
now = dt.datetime.now()
msg = EmailMessage()
msg['Subject'] = 'Holdings above 2% in accounts bigger than 300M'
msg['From'] = EMAIL_ADDRESS
msg['To'] = to_email
msg.set_content(f'Attached is a file containing holdings that are over 2% in accounts larger than 300 million, time sent: {now}')

In [20]:
with open('data_2.csv', 'rb') as file:
    file_data = file.read()
    file_name = file.name
    msg.add_attachment(file_data, maintype="application", subtype="csv", filename=file_name)

In [21]:
with smtplib.SMTP_SSL("smtp.gmail.com", 465) as connection:
    connection.login(user=EMAIL_ADDRESS, password=EMAIL_PASSWORD)
    connection.send_message(msg)
print("File has been sent")

File has been sent
